In [2]:
import pandas as pd

In [28]:
df_basic_info = pd.read_csv("../data/processed/stocks-basic-info.csv")
df_prices = pd.read_csv("../data/processed/stocks-prices.csv", parse_dates=["DATE"])
df_fundaments = pd.read_csv("../data/processed/stocks-fundaments.csv", parse_dates=["DT_INI_EXERC", "DT_FIM_EXERC"])

In [29]:
df_prices.head()

,CD_CVM,TICKER,DATE,PRICE
0,906,BBDC4,2014-10-06,8.793426
1,906,BBDC4,2014-10-07,8.735580
2,906,BBDC4,2014-10-08,8.661528
3,906,BBDC4,2014-10-09,8.830453
4,906,BBDC4,2014-10-10,8.388464


In [74]:
print(df_fundaments.shape)
df_fundaments.tail()

(20, 7)


,CD_CVM,DT_INI_EXERC,DT_FIM_EXERC,DS_CONTA,VL_CONTA,EXERC_YEAR,VL_CONTA_ROLLING_YEAR
15,1023,2023-10-01,2023-12-31,Lucro ou Prejuízo,7940448.0,2023,33165591.0
16,906,2024-01-01,2024-03-31,Lucro ou Prejuízo,4202768.0,2024,13231723.0
17,1023,2024-01-01,2024-03-31,Lucro ou Prejuízo,8680714.0,2024,34555323.0
18,906,2024-04-01,2024-06-30,Lucro ou Prejuízo,4185407.0,2024,13345865.0
19,1023,2024-04-01,2024-06-30,Lucro ou Prejuízo,9485229.0,2024,34948038.0


In [11]:
df_history = df_prices.merge(df_fundaments, how='left', left_on=["CD_CVM", "DATE"], right_on=["CD_CVM", "DT_FIM_EXERC"])
df_history[~df_history["VL_CONTA_ROLLING_YEAR"].isna()]

,CD_CVM,TICKER,DATE,PRICE,DT_INI_EXERC,DT_FIM_EXERC,DS_CONTA,VL_CONTA,EXERC_YEAR,VL_CONTA_ROLLING_YEAR
2109,906,BBDC4,2023-03-31,11.793894,2023-01-01,2023-03-31,Lucro ou Prejuízo,5473810.0,2023.0,19422251.0
2170,906,BBDC4,2023-06-30,14.969824,2023-04-01,2023-06-30,Lucro ou Prejuízo,4071265.0,2023.0,16284190.0
4598,1023,BBAS3,2023-03-31,17.840614,2023-01-01,2023-03-31,Lucro ou Prejuízo,7290982.0,2023.0,32165653.0
4659,1023,BBAS3,2023-06-30,22.752424,2023-04-01,2023-06-30,Lucro ou Prejuízo,9092514.0,2023.0,32536991.0


In [96]:
df_history = pd.DataFrame()
df_dates = pd.DataFrame({"DATE": pd.date_range(start=df_prices.iloc[0]["DATE"], end=df_prices.iloc[-1]["DATE"])})

for ticker in df_prices["TICKER"].unique():
    df_prices_filtered = df_prices[df_prices["TICKER"] == ticker]

    df_tmp = df_dates.merge(df_prices_filtered, how="left", on="DATE")
    df_tmp = df_tmp.ffill()
    df_history = pd.concat([df_history, df_tmp])
    
    
fundament_type = "Lucro ou Prejuízo"
column_name = "PROFIT"

df_fundaments_filtered = df_fundaments[df_fundaments["DS_CONTA"] == fundament_type]
df_history = df_history.merge(df_fundaments_filtered, how='left', left_on=["CD_CVM", "DATE"], right_on=["CD_CVM", "DT_FIM_EXERC"])
df_history = df_history.drop(["DS_CONTA", "DT_INI_EXERC", "DT_FIM_EXERC", "EXERC_YEAR"], axis=1)
fundament_columns = [column_name, column_name + "_ROLLING_YEAR"]

df_history = df_history.rename(columns={
    "VL_CONTA": fundament_columns[0], 
    "VL_CONTA_ROLLING_YEAR": fundament_columns[1]
    })

df_history.loc[:,fundament_columns] = df_history.groupby("CD_CVM")[fundament_columns].ffill()

df_history[df_history[column_name].isna()]

,DATE,CD_CVM,TICKER,PRICE,PROFIT,PROFIT_ROLLING_YEAR
0,2014-10-06,906.0,BBDC4,8.793426,NaN,NaN
1,2014-10-07,906.0,BBDC4,8.735580,NaN,NaN
2,2014-10-08,906.0,BBDC4,8.661528,NaN,NaN
3,2014-10-09,906.0,BBDC4,8.830453,NaN,NaN
4,2014-10-10,906.0,BBDC4,8.388464,NaN,NaN
...,...,...,...,...,...,...
6380,2022-03-26,1023.0,BBAS3,14.714496,NaN,NaN
6381,2022-03-27,1023.0,BBAS3,14.714496,NaN,NaN
6382,2022-03-28,1023.0,BBAS3,14.572767,NaN,NaN
6383,2022-03-29,1023.0,BBAS3,14.635294,NaN,NaN
